In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import pickle
from datetime import datetime

## predictionデータ

In [9]:
race_id = 202142112305
year = datetime.now().year
base_URL = "https://nar.netkeiba.com/race/shutuba.html?race_id="        
menu = "&rf=race_submenu"
r = requests.get(base_URL + str(race_id) + menu, headers={'User-agent':'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0'})
c = r.content
soup = BeautifulSoup(c,"html.parser")
#あらかじめ日付情報を取得しておく
date_info = soup.find("dl",{"id":"RaceList_DateList"}).find("dd",{"class":"Active"}).a.text
date_info=date_info.split("(")[0]
#URLが変わったことにより下記が機能しない場合があるため、その場合は旧ドメインでスクレイプんぐする
try:
    if "月" in date_info:
        date_info = str(year) + '年' + date_info
        race_date = datetime.strptime(date_info,'%Y年%m月%d日')
        race_date = race_date.timestamp()*1000
    else: #"/"の場合
        date_info = str(year) + '/' + date_info
        race_date = datetime.strptime(date_info,'%Y/%m/%d')
        race_date = race_date.timestamp()*1000
except:
    old_URL = "https://race.netkeiba.com/race/shutuba.html?race_id="
    r = requests.get(old_URL + str(race_id) + menu, headers={'User-agent':'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0'})
    c = r.content
    soup = BeautifulSoup(c,"html.parser")
    #あらかじめ日付情報を取得しておく
    date_info = soup.find("dl",{"id":"RaceList_DateList"}).find("dd",{"class":"Active"}).a.text
    date_info=date_info.split("(")[0]
    if "月" in date_info:
        date_info = str(year) + '年' + date_info
        race_date = datetime.strptime(date_info,'%Y年%m月%d日')
        race_date = race_date.timestamp()*1000
    else: #"/"の場合
        date_info = str(year) + '/' + date_info
        race_date = datetime.strptime(date_info,'%Y/%m/%d')
        race_date = race_date.timestamp()*1000
    print(f"date info transformed: {date_info},\nrace_date:{race_date}")

all_rows = soup.find_all("tr",{"class":"HorseList"})
race_name = soup.find("div",{"class":"RaceName"}).text.rstrip("\n")
race_data_1 = soup.find("div",{"class":"RaceData01"}).text.strip("\n").replace(" ","").split("/") #予測データだとフィールド情報しかない。時間、天気、馬上のコンディションは不明。直前になればわかるかも？
rd2 = soup.find("div",{"class":"RaceData02"}).find_all("span")
race_data_2 = [x.text for x in rd2]
row_list=[]

In [10]:
for item in all_rows:
    d = {}
    d["Race_id"] = race_id
    d["Race_date"] = race_date
    #Race Name
    d["Race_Name"]=race_name
    #Race data 1
    d["Race_data_1"]=race_data_1
    #Race data 2
    d["Race_data_2"]=race_data_2
    #Ranking
    d["Ranking"]=None
    #Uniform number
    d["Uni_Num"]=None
    #Horse number
    d["Hor_Num"]=None
    #Horse Name
    d["Hor_name"]=item.find("span",{"class":"HorseName"}).a.text
    #Horse sex and age
    d["Hor_sex_and_age"]=item.find("span",{"class":"Age"}).text
    #JockeyWeight
    d["JockeyWeight"]=item.find_all("td",{"class":"Txt_C"})[0].text
    #Jockey
    try:
        d["Jockey"]=item.find("span",{"class":"Jockey"}).a.text
    except:
        d["Jockey"]=item.find("span",{"class":"Jockey"}).text.strip("\n")  
        #Race Time
    d["Race_Time"]=None
    #Arrival diff
    d["Arrival_diff"]=None
    #popularity
    d["Popularity"]=None
    #単勝人気
    d["Odds_popularity"]=None
    #後3F
    d["ato_3_F"]=None
    #Corner Ranking
    d["Corner_ranking"]=None
    #厩舎1
    #厩舎1
    d["Trainer_1"]=item.find("td",{"class":"Trainer"}).span.text
    #厩舎2
    d["Trainer_2"]=item.find("td",{"class":"Trainer"}).a.text
    #HorseWeight
    if item.find("td",{"class":"Weight"}).text == "\n":
        d["Horse_weight"] = None
    else:
        d["Horse_weight"]=item.find("td",{"class":"Weight"}).text[:4]
    #HorseWeight flux
    try:
        d["Horse_weight_Flux"]=item.find("td",{"class":"Weight"}).small.text.replace("(","").replace(")","").replace("+","")
    except:
        d["Horse_weight_Flux"]=None    # print(f"each item: {d}")
    row_list.append(d)

df = pd.DataFrame(row_list)
"""##取得データの整形"""
#さらに列を分解する
try:
    df['Race_data_1'].apply(lambda x: x[0].strip().replace('発走',''))
except:
    df['start_time'] = None

df['field_type'] =df['Race_data_1'].apply(lambda x: x[1].strip())

try:
    df['weather']=df['Race_data_1'].apply(lambda x: x[2].strip().replace('天候:',''))
except:
    df['weather'] = None
try:
    df['field_condition'] =df['Race_data_1'].apply(lambda x: x[3].strip().replace('馬場:',''))
except:
    df['field_condition'] =None

df['competition_count'] = df['Race_data_2'].apply(lambda x: x[0].strip().replace('回',''))
df['venue_area'] = df['Race_data_2'].apply(lambda x: x[1].strip())
df['day_count'] = df['Race_data_2'].apply(lambda x:x[2].strip().replace('日目',''))
df['horse_class'] = df['Race_data_2'].apply(lambda x:x[3].strip())
df['number_of_horses'] = df['Race_data_2'].apply(lambda x:x[4].strip().replace('頭',''))

#*賞金データはあまり関係なさそうだからdrop
#Cleanse the follwing columns
df['Hor_sex_and_age'] = df['Hor_sex_and_age'].apply(lambda x: x.strip().replace('\n',''))
df['Jockey'] = df['Jockey'].apply(lambda x: x.strip().replace("▲","").replace("△","").replace("☆","")).replace("★","")
df['JockeyWeight'] = df['JockeyWeight'].apply(lambda x: None if x=="未定" else x)
#フィールド情報を下記の通りに分解する
df['field_length'] = df['field_type'].apply(lambda x: x[1:5])
df['field_type_1'] = df['field_type'].apply(lambda x: x[0])
df['field_type_2'] = df['field_type'].apply(lambda x: x[7:].strip().replace('(','').replace(')',''))
#不要な列を削除する（主に分化し終えたもの）
df = df.drop(['Race_data_1','Race_data_2','field_type','Corner_ranking','Arrival_diff'],axis=1)
# データタイプを一括で変更する
changed_columns = list(df.columns)
changed_columns.remove('Race_date')
for column in changed_columns:
    df[column] = pd.to_numeric(df[column],errors='ignore')
df

,Race_id,Race_date,Race_Name,Ranking,Uni_Num,Hor_Num,Hor_name,Hor_sex_and_age,JockeyWeight,Jockey,Race_Time,Popularity,Odds_popularity,ato_3_F,Trainer_1,Trainer_2,Horse_weight,Horse_weight_Flux,weather,field_condition,competition_count,venue_area,day_count,horse_class,number_of_horses,field_length,field_type_1,field_type_2
0,202142112305,1.637626e+12,埼玉県養鶏協会長賞,NaN,NaN,NaN,バールベック,牝3,54.0,藤江渉,NaN,NaN,NaN,NaN,川崎,武井和実,420,-1,晴,重,9,浦和,2,サラ系３歳 3歳,12,1500,ダ,左
1,202142112305,1.637626e+12,埼玉県養鶏協会長賞,NaN,NaN,NaN,デルマケゴン,牝3,54.0,福原杏,NaN,NaN,NaN,NaN,川崎,山崎裕也,402,-10,晴,重,9,浦和,2,サラ系３歳 3歳,12,1500,ダ,左
2,202142112305,1.637626e+12,埼玉県養鶏協会長賞,NaN,NaN,NaN,ヤマニンバルトロ,牡3,56.0,山崎誠士,NaN,NaN,NaN,NaN,川崎,山崎尋美,483,1,晴,重,9,浦和,2,サラ系３歳 3歳,12,1500,ダ,左
3,202142112305,1.637626e+12,埼玉県養鶏協会長賞,NaN,NaN,NaN,エメラルスマイル,牝3,54.0,野澤憲彦,NaN,NaN,NaN,NaN,船橋,佐々木清,437,-1,晴,重,9,浦和,2,サラ系３歳 3歳,12,1500,ダ,左
4,202142112305,1.637626e+12,埼玉県養鶏協会長賞,NaN,NaN,NaN,クリムゾンハート,牝3,54.0,町田直希,NaN,NaN,NaN,NaN,川崎,佐々木仁,473,-2,晴,重,9,浦和,2,サラ系３歳 3歳,12,1500,ダ,左
5,202142112305,1.637626e+12,埼玉県養鶏協会長賞,NaN,NaN,NaN,マウコウカミ,牝3,54.0,内田利雄,NaN,NaN,NaN,NaN,浦和,藤原智行,446,-7,晴,重,9,浦和,2,サラ系３歳 3歳,12,1500,ダ,左
6,202142112305,1.637626e+12,埼玉県養鶏協会長賞,NaN,NaN,NaN,ミュリールベッルム,牝3,54.0,酒井忍,NaN,NaN,NaN,NaN,船橋,矢野義幸,507,0,晴,重,9,浦和,2,サラ系３歳 3歳,12,1500,ダ,左
7,202142112305,1.637626e+12,埼玉県養鶏協会長賞,NaN,NaN,NaN,セイウンブイワン,牡3,56.0,国分祐仁,NaN,NaN,NaN,NaN,浦和,冨田敏男,439,-4,晴,重,9,浦和,2,サラ系３歳 3歳,12,1500,ダ,左
8,202142112305,1.637626e+12,埼玉県養鶏協会長賞,NaN,NaN,NaN,サンライズテソロ,牡3,56.0,吉留孝司,NaN,NaN,NaN,NaN,浦和,川島豊,443,2,晴,重,9,浦和,2,サラ系３歳 3歳,12,1500,ダ,左
9,202142112305,1.637626e+12,埼玉県養鶏協会長賞,NaN,NaN,NaN,オッティモ,牝3,54.0,見越彬央,NaN,NaN,NaN,NaN,浦和,小林真治,391,5,晴,重,9,浦和,2,サラ系３歳 3歳,12,1500,ダ,左


## Trainingデータ

In [ ]:
base_URL = "https://nar.netkeiba.com/race/result.html?race_id="
row_list = []
r = requests.get(base_URL + str(race_id),
                headers={'User-agent':'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0'})
c = r.content
soup = BeautifulSoup(c,"html.parser")
if len(soup.find_all("table",{"summary":"全着順"})) >0:
    #あらかじめ日付情報を取得しておく
    date_info = soup.find("dl",{"id":"RaceList_DateList"}).find("dd",{"class":"Active"}).a.text
    date_info=date_info.split("(")[0]
    if "月" in date_info:
        date_info = str(year) + '年' + date_info
        race_date = datetime.strptime(date_info,'%Y年%m月%d日')
        race_date = race_date.timestamp()*1000
    else: #"/"の場合
        date_info = str(year) + '/' + date_info
        race_date = datetime.strptime(date_info,'%Y/%m/%d')
        race_date = race_date.timestamp()*1000
    # print(f"This table is newer than the last record!:\n{race_id}")

In [ ]:
    all_rows = soup.find("tbody")
    all_rows = all_rows.find_all("tr")
    # print(all_rows)
    #そのほかの共通データ
    race_name = soup.find("div",{"class":"RaceName"}).text.rstrip("\n")
    race_data_1 = soup.find("div",{"class":"RaceData01"}).text.split("/")
    rd2 = soup.find("div",{"class":"RaceData02"}).find_all("span")
    race_data_2 = [x.text for x in rd2]
    #コーナー順位のデータ(共通)
    corner_rank_list = []
    corner_rank_table = soup.find("table",{"summary":"コーナー通過順位"})
    for corner in corner_rank_table.find_all("tr"):
        corner_rank_list.append(corner.text.replace("\n",""))    

In [ ]:
for item in all_rows:
    d = {}
    #Race id
    d["Race_id"]=race_id
    #Race date
    d["Race_date"]=race_date
    #Race Name
    d["Race_Name"]=race_name
    #Race data 1
    d["Race_data_1"]=race_data_1
    #Race data 2
    d["Race_data_2"]=race_data_2
    #Ranking
    d["Ranking"]=item.find("td",{"class":"Result_Num"}).div.text
    #Uniform number
    d["Uni_Num"]=item.find("td",{"class":"Num"}).div.text
    #Horse number
    d["Hor_Num"]=item.find("td",{"class":"Num Waku"}).div.text
    #Horse Name
    d["Hor_name"]=item.find("span",{"class":"Horse_Name"}).a.text
    #Horse sex and age
    d["Hor_sex_and_age"]=item.find("span",{"class":"Detail_Left"}).text
    #JockeyWeight
    d["JockeyWeight"]=item.find("span",{"class":"JockeyWeight"}).text
    #Jockey
    d["Jockey"]=item.find("td",{"class":"Jockey"}).a.text
    #Race Time
    d["Race_Time"]=item.find_all("span",{"class":"RaceTime"})[0].text
    #Arrival diff
    d["Arrival_diff"]=item.find_all("td",{"class":"Time"})[1].text
    #popularity
    d["Popularity"]=item.find("span",{"class":"OddsPeople"}).text
    #単勝人気
    d["Odds_popularity"]=item.find_all("td",{"class":"Odds"})[1].text
    #後3F
    d["ato_3_F"]=item.find_all("td",{"class":"Time"})[-1].text
    #Corner Ranking
    try:
        d["Corner_rank_1"]=corner_rank_list[0]
    except:
        d["Corner_rank_1"]=None
    try:
        d["Corner_rank_2"]=corner_rank_list[1]
    except:
        d["Corner_rank_2"]=None
    try:
        d["Corner_rank_3"]=corner_rank_list[2]
    except:
        d["Corner_rank_3"]=None
    try:
        d["Corner_rank_4"]=corner_rank_list[3]
    except:
        d["Corner_rank_4"]=None
    #厩舎1
    d["Trainer_1"]=item.find("td",{"class":"Trainer"}).span.text
    #厩舎2
    d["Trainer_2"]=item.find("td",{"class":"Trainer"}).a.text
    #HorseWeight
    d["Horse_weight"]=item.find("td",{"class":"Weight"}).text[:4]
    #HorseWeight flux
    d["Horse_weight_Flux"]=item.find("td",{"class":"Weight"}).small.text
    row_list.append(d)
    # print(f"craped data of {race_id}:\n{row_list}")

In [ ]:
# df = pd.DataFrame(row_list)
# df

In [ ]:
df['start_time'] = df['Race_data_1'].apply(lambda x: x[0].strip().replace('発走',''))
df['field_type'] =df['Race_data_1'].apply(lambda x: x[1].strip())
df['weather'] = df['Race_data_1'].apply(lambda x: x[2].strip().replace('天候:',''))
try:
    df['field_condition'] =df['Race_data_1'].apply(lambda x: x[3].strip().replace('馬場:',''))
except:
    df['field_condition'] = None
df['competition_count'] = df['Race_data_2'].apply(lambda x: x[0].strip().replace('回',''))
df['venue_area'] = df['Race_data_2'].apply(lambda x: x[1].strip())
df['day_count'] = df['Race_data_2'].apply(lambda x:x[2].strip().replace('日目',''))
df['horse_class'] = df['Race_data_2'].apply(lambda x:x[3].strip())
df['number_of_horses'] = df['Race_data_2'].apply(lambda x:x[4].strip().replace('頭',''))

In [ ]:
df

In [ ]:
#Cleanse the follwing columns
df['Hor_sex_and_age'] = df['Hor_sex_and_age'].apply(lambda x: x.strip().replace('\n',''))
df['Arrival_diff'] = df['Arrival_diff'].apply(lambda x: x.strip().replace('\n',''))
df['Odds_popularity'] = df['Odds_popularity'].apply(lambda x: x.strip().replace('\n',''))
df['ato_3_F'] = df['ato_3_F'].apply(lambda x: x.strip().replace('\n',''))
df['Corner_rank_1'] = df['Hor_Num'] + ',' + df['Corner_rank_1'].apply(lambda x:x[5:])
df['Corner_rank_2'] = df['Hor_Num'] + ',' + df['Corner_rank_2'].apply(lambda x:x[5:])
df['Corner_rank_3'] = df['Hor_Num'] + ',' + df['Corner_rank_3'].apply(lambda x:x[5:])
df['Corner_rank_4'] = df['Hor_Num'] + ',' + df['Corner_rank_4'].apply(lambda x:x[5:])
df['Horse_weight_Flux'] = df['Horse_weight_Flux'].apply(lambda x: x.strip().replace('(','').replace(')','').replace('+',''))
#race_dataによっては時間情報がなくslicingがうまく行かない場合があるので、下記の関数を別途定義する
def start_time_converter(x):
    try:
        start = int(x.strip().replace(':',''))
    except:
        start = None
    return start
df['start_time'] = df['start_time'].apply(start_time_converter)
df['Horse_weight'] = df['Horse_weight'].apply(lambda x: x.strip().replace(' ',''))
df['Jockey'] = df['Jockey'].apply(lambda x: x.strip().replace("▲","").replace("△","").replace("☆","").replace("◇",""))

# Race Timeをコンバートするには下記の関数を使う
def get_sec(time_str):
    """Get Seconds from time."""
    try:
        m, s = time_str.split(':')
    except:
        return None #おそらくまだ順位が出ていないデータに対して
    return  int(m) * 60 + float(s)

df['Race_Time'] = df['Race_Time'].apply(get_sec) #上記の関数を適用


In [ ]:
df

In [ ]:
#順位の列は下記の関数を使って分解する
'''
処理内容
1.各コーナー順位のリストの一番最初に馬番を追加する
2.2番目以降の数字をfor文で回して上記の数字と一致するかを調べる
3.一致する場合はその数字の順番を返り値にする
'''
def rank(rank_list):
    rank_list = rank_list.split(",")
    hor_num = rank_list[0]
    for idx,item in enumerate(rank_list):
        if idx == 0:
            pass
        else:
            if item == hor_num:
                return idx

In [ ]:
df['Corner_rank_1'] = df['Corner_rank_1'].apply(rank)
df['Corner_rank_2'] = df['Corner_rank_2'].apply(rank)
df['Corner_rank_3'] = df['Corner_rank_3'].apply(rank)
df['Corner_rank_4'] = df['Corner_rank_4'].apply(rank)

In [ ]:
df

,Race_id,Race_date,Race_Name,Race_data_1,Race_data_2,Ranking,Uni_Num,Hor_Num,Hor_name,Hor_sex_and_age,JockeyWeight,Jockey,Race_Time,Arrival_diff,Popularity,Odds_popularity,ato_3_F,Corner_rank_1,Corner_rank_2,Corner_rank_3,Corner_rank_4,Trainer_1,Trainer_2,Horse_weight,Horse_weight_Flux,start_time,field_type,weather,field_condition,competition_count,venue_area,day_count,horse_class,number_of_horses
0,202155111303,1.636762e+12,木枯らし賞,"[\n16:30発走 , ダ1300m (右)\n, 天候:曇\n, 馬場:稍\n \n]","[15回, 佐賀, 2日目, サラ系３歳以上 C2, 11頭, 本賞金:30.0、9.0、4...",1,7,8,アスクローンウルフ,牡3,56.0,倉富隆一,84.5,,5,20.8,39.9,2,2,2,1,佐賀,川田孝好,480,7,1630,ダ1300m (右),曇,稍,15,佐賀,2,サラ系３歳以上 C2,11
1,202155111303,1.636762e+12,木枯らし賞,"[\n16:30発走 , ダ1300m (右)\n, 天候:曇\n, 馬場:稍\n \n]","[15回, 佐賀, 2日目, サラ系３歳以上 C2, 11頭, 本賞金:30.0、9.0、4...",2,4,4,カシノオラフ,牡3,56.0,石川慎将,84.5,クビ,3,15.4,38.5,9,8,7,6,佐賀,古賀光範,476,-2,1630,ダ1300m (右),曇,稍,15,佐賀,2,サラ系３歳以上 C2,11
2,202155111303,1.636762e+12,木枯らし賞,"[\n16:30発走 , ダ1300m (右)\n, 天候:曇\n, 馬場:稍\n \n]","[15回, 佐賀, 2日目, サラ系３歳以上 C2, 11頭, 本賞金:30.0、9.0、4...",3,8,10,ピヨピヨピピピ,牝3,54.0,飛田愛斗,84.9,2,2,2.8,40.1,3,3,3,2,佐賀,三小田幸,437,3,1630,ダ1300m (右),曇,稍,15,佐賀,2,サラ系３歳以上 C2,11
3,202155111303,1.636762e+12,木枯らし賞,"[\n16:30発走 , ダ1300m (右)\n, 天候:曇\n, 馬場:稍\n \n]","[15回, 佐賀, 2日目, サラ系３歳以上 C2, 11頭, 本賞金:30.0、9.0、4...",4,8,11,サンマルファミリー,牝3,51.0,加茂飛翔,85.1,3/4,4,18.7,39.4,5,5,5,4,佐賀,大島静夫,468,-9,1630,ダ1300m (右),曇,稍,15,佐賀,2,サラ系３歳以上 C2,11
4,202155111303,1.636762e+12,木枯らし賞,"[\n16:30発走 , ダ1300m (右)\n, 天候:曇\n, 馬場:稍\n \n]","[15回, 佐賀, 2日目, サラ系３歳以上 C2, 11頭, 本賞金:30.0、9.0、4...",5,5,5,モンテモナコ,牝3,54.0,出水拓人,85.3,1.1/4,6,25.3,39.9,4,4,4,5,佐賀,古賀光範,439,-6,1630,ダ1300m (右),曇,稍,15,佐賀,2,サラ系３歳以上 C2,11
5,202155111303,1.636762e+12,木枯らし賞,"[\n16:30発走 , ダ1300m (右)\n, 天候:曇\n, 馬場:稍\n \n]","[15回, 佐賀, 2日目, サラ系３歳以上 C2, 11頭, 本賞金:30.0、9.0、4...",6,7,9,エイシンアルゴス,牡4,55.0,金山昇馬,85.5,1,1,1.5,39.7,8,7,6,7,佐賀,北村欣也,490,2,1630,ダ1300m (右),曇,稍,15,佐賀,2,サラ系３歳以上 C2,11
6,202155111303,1.636762e+12,木枯らし賞,"[\n16:30発走 , ダ1300m (右)\n, 天候:曇\n, 馬場:稍\n \n]","[15回, 佐賀, 2日目, サラ系３歳以上 C2, 11頭, 本賞金:30.0、9.0、4...",7,3,3,ミスミシマ,牝3,54.0,村松翔太,85.6,3/4,11,339.0,39.7,6,6,8,8,佐賀,柳井宏之,435,0,1630,ダ1300m (右),曇,稍,15,佐賀,2,サラ系３歳以上 C2,11
7,202155111303,1.636762e+12,木枯らし賞,"[\n16:30発走 , ダ1300m (右)\n, 天候:曇\n, 馬場:稍\n \n]","[15回, 佐賀, 2日目, サラ系３歳以上 C2, 11頭, 本賞金:30.0、9.0、4...",8,6,7,ナレディ,牝5,54.0,川島拓,85.8,3/4,10,255.3,39.2,7,9,10,10,佐賀,土井道隆,395,0,1630,ダ1300m (右),曇,稍,15,佐賀,2,サラ系３歳以上 C2,11
8,202155111303,1.636762e+12,木枯らし賞,"[\n16:30発走 , ダ1300m (右)\n, 天候:曇\n, 馬場:稍\n \n]","[15回, 佐賀, 2日目, サラ系３歳以上 C2, 11頭, 本賞金:30.0、9.0、4...",9,2,2,ダイメイスイセン,牝4,54.0,山下裕貴,86.2,2,9,65.0,39.4,11,10,9,9,佐賀,大垣敏夫,423,-7,1630,ダ1300m (右),曇,稍,15,佐賀,2,サラ系３歳以上 C2,11
9,202155111303,1.636762e+12,木枯らし賞,"[\n16:30発走 , ダ1300m (右)\n, 天候:曇\n, 馬場:稍\n \n]","[15回, 佐賀, 2日目, サラ系３歳以上 C2, 11頭, 本賞金:30.0、9.0、4...",10,1,1,エイシンリベルタス,牝3,54.0,兒島真二,87.1,5,8,64.4,42.6,1,1,1,3,佐賀,三小田幸,469,-2,1630,ダ1300m (右),曇,稍,15,佐賀,2,サラ系３歳以上 C2,11


In [ ]:
df['field_length'] = df['field_type'].apply(lambda x: None if '候' in x else x[1:5])
df['field_type_1'] = df['field_type'].apply(lambda x: x[0])
df['field_type_2'] = df['field_type'].apply(lambda x: x[7:].strip().replace('(','').replace(')',''))
df

,Race_id,Race_date,Race_Name,Race_data_1,Race_data_2,Ranking,Uni_Num,Hor_Num,Hor_name,Hor_sex_and_age,JockeyWeight,Jockey,Race_Time,Arrival_diff,Popularity,Odds_popularity,ato_3_F,Corner_rank_1,Corner_rank_2,Corner_rank_3,Corner_rank_4,Trainer_1,Trainer_2,Horse_weight,Horse_weight_Flux,start_time,field_type,weather,field_condition,competition_count,venue_area,day_count,horse_class,number_of_horses,field_length,field_type_1,field_type_2
0,202155111303,1.636762e+12,木枯らし賞,"[\n16:30発走 , ダ1300m (右)\n, 天候:曇\n, 馬場:稍\n \n]","[15回, 佐賀, 2日目, サラ系３歳以上 C2, 11頭, 本賞金:30.0、9.0、4...",1,7,8,アスクローンウルフ,牡3,56.0,倉富隆一,84.5,,5,20.8,39.9,2,2,2,1,佐賀,川田孝好,480,7,1630,ダ1300m (右),曇,稍,15,佐賀,2,サラ系３歳以上 C2,11,1300,ダ,右
1,202155111303,1.636762e+12,木枯らし賞,"[\n16:30発走 , ダ1300m (右)\n, 天候:曇\n, 馬場:稍\n \n]","[15回, 佐賀, 2日目, サラ系３歳以上 C2, 11頭, 本賞金:30.0、9.0、4...",2,4,4,カシノオラフ,牡3,56.0,石川慎将,84.5,クビ,3,15.4,38.5,9,8,7,6,佐賀,古賀光範,476,-2,1630,ダ1300m (右),曇,稍,15,佐賀,2,サラ系３歳以上 C2,11,1300,ダ,右
2,202155111303,1.636762e+12,木枯らし賞,"[\n16:30発走 , ダ1300m (右)\n, 天候:曇\n, 馬場:稍\n \n]","[15回, 佐賀, 2日目, サラ系３歳以上 C2, 11頭, 本賞金:30.0、9.0、4...",3,8,10,ピヨピヨピピピ,牝3,54.0,飛田愛斗,84.9,2,2,2.8,40.1,3,3,3,2,佐賀,三小田幸,437,3,1630,ダ1300m (右),曇,稍,15,佐賀,2,サラ系３歳以上 C2,11,1300,ダ,右
3,202155111303,1.636762e+12,木枯らし賞,"[\n16:30発走 , ダ1300m (右)\n, 天候:曇\n, 馬場:稍\n \n]","[15回, 佐賀, 2日目, サラ系３歳以上 C2, 11頭, 本賞金:30.0、9.0、4...",4,8,11,サンマルファミリー,牝3,51.0,加茂飛翔,85.1,3/4,4,18.7,39.4,5,5,5,4,佐賀,大島静夫,468,-9,1630,ダ1300m (右),曇,稍,15,佐賀,2,サラ系３歳以上 C2,11,1300,ダ,右
4,202155111303,1.636762e+12,木枯らし賞,"[\n16:30発走 , ダ1300m (右)\n, 天候:曇\n, 馬場:稍\n \n]","[15回, 佐賀, 2日目, サラ系３歳以上 C2, 11頭, 本賞金:30.0、9.0、4...",5,5,5,モンテモナコ,牝3,54.0,出水拓人,85.3,1.1/4,6,25.3,39.9,4,4,4,5,佐賀,古賀光範,439,-6,1630,ダ1300m (右),曇,稍,15,佐賀,2,サラ系３歳以上 C2,11,1300,ダ,右
5,202155111303,1.636762e+12,木枯らし賞,"[\n16:30発走 , ダ1300m (右)\n, 天候:曇\n, 馬場:稍\n \n]","[15回, 佐賀, 2日目, サラ系３歳以上 C2, 11頭, 本賞金:30.0、9.0、4...",6,7,9,エイシンアルゴス,牡4,55.0,金山昇馬,85.5,1,1,1.5,39.7,8,7,6,7,佐賀,北村欣也,490,2,1630,ダ1300m (右),曇,稍,15,佐賀,2,サラ系３歳以上 C2,11,1300,ダ,右
6,202155111303,1.636762e+12,木枯らし賞,"[\n16:30発走 , ダ1300m (右)\n, 天候:曇\n, 馬場:稍\n \n]","[15回, 佐賀, 2日目, サラ系３歳以上 C2, 11頭, 本賞金:30.0、9.0、4...",7,3,3,ミスミシマ,牝3,54.0,村松翔太,85.6,3/4,11,339.0,39.7,6,6,8,8,佐賀,柳井宏之,435,0,1630,ダ1300m (右),曇,稍,15,佐賀,2,サラ系３歳以上 C2,11,1300,ダ,右
7,202155111303,1.636762e+12,木枯らし賞,"[\n16:30発走 , ダ1300m (右)\n, 天候:曇\n, 馬場:稍\n \n]","[15回, 佐賀, 2日目, サラ系３歳以上 C2, 11頭, 本賞金:30.0、9.0、4...",8,6,7,ナレディ,牝5,54.0,川島拓,85.8,3/4,10,255.3,39.2,7,9,10,10,佐賀,土井道隆,395,0,1630,ダ1300m (右),曇,稍,15,佐賀,2,サラ系３歳以上 C2,11,1300,ダ,右
8,202155111303,1.636762e+12,木枯らし賞,"[\n16:30発走 , ダ1300m (右)\n, 天候:曇\n, 馬場:稍\n \n]","[15回, 佐賀, 2日目, サラ系３歳以上 C2, 11頭, 本賞金:30.0、9.0、4...",9,2,2,ダイメイスイセン,牝4,54.0,山下裕貴,86.2,2,9,65.0,39.4,11,10,9,9,佐賀,大垣敏夫,423,-7,1630,ダ1300m (右),曇,稍,15,佐賀,2,サラ系３歳以上 C2,11,1300,ダ,右
9,202155111303,1.636762e+12,木枯らし賞,"[\n16:30発走 , ダ1300m (右)\n, 天候:曇\n, 馬場:稍\n \n]","[15回, 佐賀, 2日目, サラ系３歳以上 C2, 11頭, 本賞金:30.0、9.0、4...",10,1,1,エイシンリベルタス,牝3,54.0,兒島真二,87.1,5,8,64.4,42.6,1,1,1,3,佐賀,三小田幸,469,-2,1630,ダ1300m (右),曇,稍,15,佐賀,2,サラ系３歳以上 C2,11,1300,ダ,右


## そのほかいろいろ試す

In [18]:
with open('/content/drive/MyDrive/MyApps/HorseRacingApp/app_azure_realtime/test_df.pkl','rb') as f:
    df = pickle.load(f)
print(len(df))

58


In [4]:
df = pd.read_html('https://race.netkeiba.com/race/result.html?race_id=202105050801&rf=race_list')

In [6]:
len(df)

6

IndexError: ignored